In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np

### 데이터 불러오기

In [15]:
b=pd.read_csv(r'C:\Users\user\OneDrive\바탕화~1-DESKTOP-R00ORLS-348\캡스톤\data_git\data\diningcode_data\다이닝코드 북촌 리뷰 분리.csv')


In [16]:
b.head()

,상가업소번호,가게명,원래이름,리뷰,분리리뷰,리뷰날짜
0,20592276,깡통만두,깡통만두,"만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.칼만두국을 먹었는데 사골베이스에 ...","만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.",2023-3-9
1,20592276,깡통만두,깡통만두,"만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.칼만두국을 먹었는데 사골베이스에 ...",칼만두국을 먹었는데 사골베이스에 조금 슴슴한 편인데 맛이 꽤 훌륭하다.,2023-3-9
2,20592276,깡통만두,깡통만두,"만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.칼만두국을 먹었는데 사골베이스에 ...",면발은 하늘하늘 부드러운 편이다.,2023-3-9
3,20592276,깡통만두,깡통만두,"만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.칼만두국을 먹었는데 사골베이스에 ...",테이블 간격이 좁고 서버 이모님들이 살짝 불친절한 편이라 온전히 유명세의 맛경험을 ...,2023-3-9
4,20592276,깡통만두,깡통만두,주말에 방문하면 웨이팅이 깁니다. 한시간정도 대기하면 드실 수 있어요. 만두가 깔끔...,주말에 방문하면 웨이팅이 깁니다.,2023-2-19


In [17]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상가업소번호  3142 non-null   int64 
 1   가게명     3142 non-null   object
 2   원래이름    3142 non-null   object
 3   리뷰      3142 non-null   object
 4   분리리뷰    3142 non-null   object
 5   리뷰날짜    3142 non-null   object
dtypes: int64(1), object(5)
memory usage: 147.4+ KB


In [38]:
for i in range(len(b)):
    if len(b['분리리뷰'][i])<3:
        print(i)
        print(b['분리리뷰'][i])

46
음.
2297
꼭
2679
추천


In [35]:
b.drop([1082,1550,1784,1904,3139],axis=0,inplace=True)


In [37]:
b=b.reset_index(drop=True)

### 불용어 제거

In [39]:
# review 컬럼만 리스트로 저장
text=''
reviews=[]
for each_review in b['분리리뷰']:
    reviews.append(each_review)

In [40]:
len(reviews)

3137

In [41]:
#불용어 제거
import re
def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    return text   

review=[]
for i in reviews:  #문자가 들어있을때는 인덱스 사용하면 안됨!!!!
    a=clean_str(i)
    review.append(a)  #불용어제거한 review 저장

In [42]:
review[:10]

['만두가 옛날 집에서 빚은 딱 그 맛이어서 좋았다',
 '칼만두국을 먹었는데 사골베이스에 조금 슴슴한 편인데 맛이 꽤 훌륭하다',
 '면발은 하늘하늘 부드러운 편이다',
 '테이블 간격이 좁고 서버 이모님들이 살짝 불친절한 편이라 온전히 유명세의 맛경험을 하기엔 다소 아쉬움이 있다',
 '주말에 방문하면 웨이팅이 깁니다',
 '한시간정도 대기하면 드실 수 있어요',
 '만두가 깔끔하고 맛있네요',
 '테이블 간 간격이 좁아서 후다닥 먹고 나왔네요 ',
 '직접 빚은 만두와 직접 뽑은 생면이 있는 안국역 수요미식회 맛집 깡통만두',
 '만두전골을 시키면 고기만두 해물만두 두 종류를 주는데 맛있고 해물만두는 새우한마리가 통으로 들어있다']

In [56]:
for i in range(len(review)):
    if review[i]=='':
        print(i)

In [66]:
review[347]

'흠'

### 키워드 추출 n_gram_range(1,1)

In [61]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 키워드 길이 지정
n_gram_range = (1,2)

In [67]:
#리뷰 하나씩 키워드로 분리(TFidf=Vectorizer)

a= len(review) # 불용어 제거한 리뷰
token_review_list=[]

for i in range(348,a): # 리뷰하나당 처리하기위해 for문 
    print(i)
    review_vectorized = TfidfVectorizer(ngram_range=n_gram_range).fit([review[i]])
    token_review = review_vectorized.get_feature_names_out()
    
    token_review_list.append(token_review) # 하나의 리스트를 만들어서 df에 추가해야함.
b['review_tfidf']=token_review_list # 토큰 단위로 나누어진 리뷰저장

348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [53]:
# 리뷰하나씩 키워드로 분리(count-vectorizer)
a= len(review) # 불용어 제거한 리뷰

token_review_list=[]

for i in range(0,a): # 리뷰하나당 처리하기위해 for문 
    review_vectorized = CountVectorizer(ngram_range=n_gram_range).fit([review[i]])
    token_review = review_vectorized.get_feature_names_out()
    
    token_review_list.append(token_review) # 하나의 리스트를 만들어서 df에 추가해야함.

b['review_count']=token_review_list # 토큰 단위로 나누어진 리뷰저장

ValueError: empty vocabulary; perhaps the documents only contain stop words